# K-Nearest Neighbors Classification Demo

K-nearest neighbors classification uses the labels of neighborhoods around data samples to classify unseen data samples. 

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input. 

For information on converting your dataset to cuDF format, refer to the cuDF documentation: https://docs.rapids.ai/api/cudf/stable

For additional information on cuML's Nearest Neighbors implementation: https://rapidsai.github.io/projects/cuml/en/stable/api.html#nearest-neighbors-classification

In [ ]:
import os

import numpy as np

from sklearn.datasets import make_blobs

import pandas as pd
import cudf as gd

from sklearn.neighbors import KNeighborsClassifier as skKNC
from cuml.neighbors import KNeighborsClassifier as cumlKNC

## Define Parameters

In [ ]:
n_samples = 2**17
n_features = 40

n_query = 5000

n_neighbors = 4

## Generate Data

### Host

In [ ]:
%%time
X_host_train, y_host_train = make_blobs(
   n_samples=n_samples, n_features=n_features, centers=5, random_state=0)

X_host_train = pd.DataFrame(X_host_train)
y_host_train = pd.DataFrame(y_host_train)

In [ ]:
%%time
X_host_test, y_host_test = make_blobs(
   n_samples=n_query, n_features=n_features, centers=5, random_state=0)

X_host_test = pd.DataFrame(X_host_test)
y_host_test = pd.DataFrame(y_host_test)

### Device

In [ ]:
X_device_train = gd.DataFrame.from_pandas(X_host_train)
y_device_train = gd.DataFrame.from_pandas(y_host_train)

In [ ]:
X_device_test = gd.DataFrame.from_pandas(X_host_test)
y_device_test = gd.DataFrame.from_pandas(y_host_test)

## Scikit-learn Model

In [ ]:
%%time
knn_sk = skKNC(algorithm="brute", n_neighbors=n_neighbors, n_jobs=-1)
knn_sk.fit(X_host_train, y_host_train)

sk_result = knn_sk.predict(X_host_test)

## cuML Model

In [ ]:
%%time
knn_cuml = cumlKNC(n_neighbors=n_neighbors)
knn_cuml.fit(X_device_train, y_device_train)

cuml_result = knn_cuml.predict(X_device_test)

## Compare Results

In [ ]:
passed = np.array_equal(np.asarray(cuml_result.as_gpu_matrix())[:,0], sk_result)
print('compare knn: cuml vs sklearn classes %s'%('equal'if passed else 'NOT equal'))